In [1]:
import time
from datetime import datetime, timedelta
import os

from Utils.class_files import Chatbot, Utils

from googletrans import Translator
translator = Translator()

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
#browser + utils
utils = Utils(language='pt')

#Chatbot
chatBotClass = Chatbot(botIdName = 'HinataBot1'
                 ,botName = 'Hinata Hyuga'
                 ,fileName = 'hinata.txt'
                 ,streamId= 'HinataBot1'
                 ,pronouns = 'feminino'
                 ,mainUser = 'odeiomodernismo'
                 
                 ,utils_class = utils)
context = chatBotClass.get_context_prompt()

In [3]:
utils.browser.get(f"https://www.twitch.tv/{chatBotClass.streamId}")
time.sleep(4)

#procedures.click_follow_button(browser) #follow
utils.click_enter_chat_offline() #entrar no chat offline
utils.click_enter_chat() #clica no chat
utils.click_skip_presentation()  #pular apresentação
utils.click_agree_rules()  #concordar com regras

print("Online!\n")

last_message = ''
chat_history_ids = []
chat_hist_debug = []

end=0
while True:
    now = (datetime.now() - timedelta(hours=0, minutes=1)).strftime("%H:%M")
    messages = utils.browser.find_elements(By.CLASS_NAME, "chat-line__message")

    for messageId in messages:
        try:
            mTime, mUser, mText = utils.get_time_user_message(messageId)
        except: pass

        if (chatBotClass.botIdName != mUser) and (f"{chatBotClass.botIdName.lower()}" in mText.lower()) and (mText!=last_message) and (messageId not in chat_history_ids) and (mTime>=now):
            chat_history_ids.append(messageId)
            chat_hist_debug.append(messageId.text)

            if ("!quit" in mText) and (mUser == f'{chatBotClass.mainUser}'):
                end=1
                break

            else:
                inputText = mText.replace(f'@{chatBotClass.botIdName}','')
                print(f'{mTime} | {mUser}: {mText}')
                chat_output = chatBotClass.chatbot.chat(context + f"mensagem de {mUser}: " + inputText)
                chat_output_raw = chat_output

                chat_output = translator.translate(chat_output, src='en', dest='pt').text
                chat_output = str(chat_output).replace('"','')

                chat_output = chatBotClass.check_output_length(chat_output, 250)
                chat_output = chatBotClass.check_pronouns(chat_output)

                chat_output = ''.join(c for c in chat_output if c <= '\uFFFF')
                chat_output = f'@{mUser} ' + chat_output

                print(f'      | Bot: {chat_output_raw}\n')
                chat_hist_debug.append(chat_output)
                utils.browser.find_element('xpath','//*[@id="WYSIWGChatInputEditor_SkipChat"]/div/div[2]').send_keys(chat_output)
                utils.actions.send_keys(Keys.ENTER)
                utils.actions.perform()
        
        last_message = mText
    
    if end==1:
        break
    time.sleep(1)

Online!

08:25 | odeiomodernismo: @hinatabot1 oii
      | Bot:  Hey there! I'm Hinata Hyuga, a gentle and determined woman from the Hidden Leaf Village. I'm a big fan of the New Jeans K-pop group because they're so cute! I'm also a Christian and my faith is very important to me. I love watching romantic movies and reading books set in the 19th and 18th centuries. And of course, I can't forget about my beloved nandinha! 🐰❤️

08:26 | odeiomodernismo: oii @hinatabot1
      | Bot:  Hi! I'm Hinata Hyuga, a kind and determined woman from the Hidden Leaf Village. I'm a big fan of the New Jeans K-pop group because they're so cute! I'm also a Christian and my faith is very important to me. I love watching romantic movies and reading books set in the 19th and 18th centuries. And of course, I can't forget about my beloved nandinha! 🐰❤️

08:26 | odeiomodernismo: o que você acha do Jean @HinataBot1 ?
      | Bot:  Oh, eu adoro o Jean! Ele é tão fofo e carinhoso, Always making me smile. Eu poderia f

In [ ]:
for i in chat_hist_debug:
    print(i)

Este é o histórico do chat, use-o para lembrar as últimas mensagens dos usuários:
mensagem de odeiomodernismo: fala pra gente o que você acha da rytsa? 
mensagem de Rytsa Fanboy: Rytsa é a melhor, fofa e talentosa!
mensagem de odeiomodernismo: se apresenta por favor 
mensagem de Rytsa Fanboy: Rytsa Fanboy, aqui presente! 

